In [146]:
import spacy
import neuralcoref
import textacy
import string
import pandas as pd
# Initializing spacy 'en' model
nlp = spacy.load('en_core_web_sm')
neuralcoref.add_to_pipe(nlp)

In [147]:
# Loading in json
import json
with open('new_competitors.json') as json_file:
    data = json.load(json_file)

In [148]:
#create company list
company_list = ['Cisco Systems','VMware', 'Big Switch Networks','D-Link','Extreme Networks','Arista Networks',
                'Hewlett Packard Enterprise','Juniper Networks','Huawei','Netgear','Cumulus Networks','Allied Telesis',
                'NEC','Mellanox Technologies','Dell EMC','ZTE','Lenovo']
var_list = company_list.copy()
for i in range(len(company_list)):
    var_list[i] = var_list[i].replace(' ', '_')
    var_list[i] = var_list[i].replace('-', '_')

In [149]:
#create dictionary
company_sep_dict = {}
company_full_text_dict = {}
for i in range(len(company_list)):
    for j in range(len(data)):
        if data[j]['Title'] == company_list[i]:
            company_sep_dict[var_list[i]] = data[j]['strings']
            company_full_text_dict[var_list[i]] = [''.join(data[j]['strings'][:])]

In [150]:
# use neuralcoref for clustering topics and use subject_verb_object_triples for services identification
no_sep = ['D_Link', 'Juniper_Networks', 'Big_Switch_Networks', 'Cumulus_Networks']
company_services = {}
for i in var_list:
    # get company strings
    sep_text = company_sep_dict[i]
    full_text = company_full_text_dict[i]
    
    doc_coref = list()
    doc_sub_v_sub = list()
    
    find_ = i.find('_')
    if find_:
        if any(x in i for x in no_sep) != True:
            sub_list = [i[:find_], i.replace('_', ' ')]
        elif i == 'D_Link':
            sub_list = [i.replace('_', '-')]
    else:
        sub_list = [i.replace('_', ' ')]
    
    for j in range(len(sep_text)):
    
        # finding all coref clusters
        doc = nlp(sep_text[j])
        if doc._.has_coref:
            for k in doc._.coref_clusters[0]:
                if any(x in str(k) for x in sub_list):
                    x = True
                else:
                    x = False
            if x:
                for z in doc._.coref_clusters[0]:
                    doc_coref.append(str(z))
        
        doc_coref.append(sub_list[0])
        doc_coref = list(set(doc_coref))
        doc_sub_v_sub.append(list(textacy.extract.subject_verb_object_triples(doc)))
        
    service_list = ['develop', 'produce', 'create', 'make', 'manufacture', 
                 'sell', 'provide', 'markets']
    
    services = list()
    for j in doc_sub_v_sub:
        if j:
            for k in range(len(j)):
                for z in service_list:
                    if z in str(j[k][1]):
                        if any(x in str(j[k][0]) for x in doc_coref):
                            services.append(z + ' ' + str(j[k][2]))
    company_services[i] = services
company_services['Lenovo'] = []

In [151]:
# Find products using Spacy label_
not_product = ['Ethernet','Virtual', 'Windows', 'D-Link', 'Bechtolsheim', 
               'Google', 'Eye-Comfort', 'HK$2.07', 'services', '2012']
for i in var_list:
    doc = nlp(company_full_text_dict[i][0])
    temp = ['produces ' + X.text for X in doc.ents if X.label_ == 'PRODUCT' 
                               and X.text not in not_product]
    company_services[i].extend(temp)

In [152]:
company_services

{'Cisco_Systems': ['develop ASICs',
  'develop processing cards',
  'provide IT products',
  'provide services',
  'provide training',
  'produces Catalyst 6500',
  'produces CRS-1',
  'produces ASR9000',
  'produces EZChip',
  'produces Webex',
  'produces OpenDNS',
  'produces Spherix'],
 'VMware': ['provide cloud computing',
  'provide virtualization software',
  'provide services',
  'provide set',
  'produces Virtual SMP',
  'produces Cloud Foundry'],
 'Big_Switch_Networks': [],
 'D_Link': [],
 'Extreme_Networks': ['produces Networld+Interop'],
 'Arista_Networks': ['develop module', 'produces Gigabit Ethernet'],
 'Hewlett_Packard_Enterprise': [],
 'Juniper_Networks': ['produces ScreenOS', 'produces EX4200'],
 'Huawei': ['create network',
  'sell xDSL',
  'provide telecommunications operators',
  'produces C&C08',
  'produces H3C',
  'produces C300'],
 'Netgear': ['provide lifetime warranties',
  'markets network appliances',
  'sell premium NAS devices',
  'create Arlo',
  'produc

# Create Node, Edge, and Connection Tables

In [153]:
node_properties= pd.DataFrame(columns=['Node_Primary_Key','Node_Type_Key','Property_Name','Property'])
node_properties

,Node_Primary_Key,Node_Type_Key,Property_Name,Property


In [154]:
# creating edge properties table for Neo4j insertion
edge_properties = pd.DataFrame(columns = ['Edge_Primary_Key','Edge_Type_Key','Property_Name','Property'])
edge_properties

,Edge_Primary_Key,Edge_Type_Key,Property_Name,Property


In [155]:
# creating connections table for Neo4j insertion
connections= pd.DataFrame(columns=['Node_Property_Key','Edge_Property_Key','Node_Property_Key_2'])

In [156]:
count = 0
service_list = ['develop', 'create', 'make', 'manufacture', 
                 'sell', 'provide', 'markets']
for i in range(len(var_list)):
    temp = company_services[var_list[i]]
    service = 0
    product = 0
    if temp:
        for j in range(len(temp)):
            space = temp[j].find(' ')
            
            if any(x in temp[j][:space] for x in service_list):
                prop = 8
                
            elif ('produces' in temp[j][:space]) or ('produce' in temp[j][:space]):
                prop = 7
            
            t = {'Node_Primary_Key': [count], 'Node_Type_Key': [prop], 
                 'Property_Name' : ['Name'], 'Property' : [temp[j][space+1:]]}

            temp_node = pd.DataFrame(data = t)
            node_properties = node_properties.append(temp_node, ignore_index = True)

            e = {'Edge_Primary_Key': [count], 'Edge_Type_Key': [8], 
                 'Property_Name': ['Found_in'], 'Property': ['Wikepedia_text']}
            temp_edge = pd.DataFrame(data = e)
            edge_properties = edge_properties.append(temp_edge, ignore_index = True)
        
            t = {'Node_Property_Key': [i], 'Edge_Property_Key': [count], 
                 'Node_Property_Key_2' : [count]}
            temp_conn = pd.DataFrame(data = t)
            connections = connections.append(temp_conn, ignore_index = True)
            count += 1

In [157]:
node_properties

,Node_Primary_Key,Node_Type_Key,Property_Name,Property
0,0,8,Name,ASICs
1,1,8,Name,processing cards
2,2,8,Name,IT products
3,3,8,Name,services
4,4,8,Name,training
5,5,7,Name,Catalyst 6500
6,6,7,Name,CRS-1
7,7,7,Name,ASR9000
8,8,7,Name,EZChip
9,9,7,Name,Webex


In [158]:
edge_properties

,Edge_Primary_Key,Edge_Type_Key,Property_Name,Property
0,0,8,Found_in,Wikepedia_text
1,1,8,Found_in,Wikepedia_text
2,2,8,Found_in,Wikepedia_text
3,3,8,Found_in,Wikepedia_text
4,4,8,Found_in,Wikepedia_text
5,5,8,Found_in,Wikepedia_text
6,6,8,Found_in,Wikepedia_text
7,7,8,Found_in,Wikepedia_text
8,8,8,Found_in,Wikepedia_text
9,9,8,Found_in,Wikepedia_text


In [159]:
connections

,Node_Property_Key,Edge_Property_Key,Node_Property_Key_2
0,0,0,0
1,0,1,1
2,0,2,2
3,0,3,3
4,0,4,4
5,0,5,5
6,0,6,6
7,0,7,7
8,0,8,8
9,0,9,9


In [160]:
edge_properties.to_pickle("services_edge_properties.pkl")
node_properties.to_pickle("services_node_properties.pkl")
connections.to_pickle("services_conn_properties.pkl")